# 네이버 웹툰 크롤링

In [6]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

driver = webdriver.Chrome()
driver.get("https://comic.naver.com/webtoon?tab=tue")

time.sleep(3)

soup = BeautifulSoup(driver.page_source, 'html.parser')
titles = soup.select(".item .ContentTitle__title--e3qXt")
links = soup.select('.item  a.ContentTitle__title_area--x24vt')
authors = soup.select('.item  .ContentAuthor__author--CTAAP')

title_list = [t.text.strip() for t in titles]
link_list = ['https://comic.naver.com' + l.get('href', '') for l in links]
author_list = [a.text.strip() for a in authors]

print(title_list)
print(link_list)
print(author_list)

driver.quit()

['괴력 난신', '김부장', '우렉 마지노', '절대회귀', '서울 자가에 대기업 다니는 김 부장 이야기', '신입사원 김철수', '이섭의 연애', '마음의소리(였던 것)', '중증외상센터 : 외과의사 백강혁', '레벨999 고블린', '마음의소리2', '축살', '사신소년', '44교시 생존수업', '멸망 이후의 세계', '회귀한 용병은 다 계획이 있다', '시한부인 줄 알았어요!', '천마는 평범하게 살 수 없다', '못 잡아먹어서 안달', '한림체육관', '아카데미 플레이어를 죽였다', '성검전설', '백씨세가 시한부 공자', '하북팽가 막내아들', '내 남편의 정부에게', '귀혼', '신과함께 돌아온 기사왕님', '체탐자', '아포칼립스에 물류센터를 숨김', '사람의 탈', '사변괴담', '긁지 않은 복권', '염라강림', '원주민 공포만화', '백작가의 비밀스런 시녀님', '좀비떼로', '개집사', '335KM', '사표내고 이계에서 힐링합니다', '저 그런 인재 아닙니다', '은주의 방 2~3부', '윈터브리즈', '천만청춘', '마교 교주의 부군이 되었다', '귀공자', '윌유메리미', '연우의 순정', '나만 보는 탑 공략집', '죽어 천국에 가다', '한계 찢는 천재마법사', '가공낙원', '왕게임', '빌런투킬', '청춘 러브썸', '야만의 청혼', '아랫집 윗집 사이에', '대공님의 신부는 지옥의 전사', '그냥 선생님', '처음도 아닌데', '킬링필드', '흑역사 어게인', '에이머', '디펜스 브레이커', '티키타카', '재난은 늘 곁에 있다', '내남친 킹카만들기', '집착남주의 전부인이 되었습니다', '인생내꺼', '선 넘는 결혼생활', '렌탈히어로', '가짜 성녀를 지켜라!', '서울역 바바리안', '무원야담', '아름다운 줄리엣을 위하여', '아카데미가 망했다', '완벽한 실눈 악역을 연기하다', '원시인 김동우', '연애 못하면 사망', '붉은 이정표', '구룡:사로카', '자멸기관', '빛나는 나나나나', '멀리서 온 에일리

In [7]:
import pandas as pd

In [ ]:
driver = webdriver.Chrome()

summaries = []
images = []
age_ratings = []

for link in link_list:
    try:
        driver.get(link)
        time.sleep(2)  # 로딩 대기
        
        if "nid.naver.com/nidlogin.login" in driver.current_url:
            print(f"성인 웹툰 감지됨: {link}")
            summaries.append("성인")
            images.append("성인")
            age_ratings.append("성인")
            continue

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 줄거리
        summary_tag = soup.select_one('.EpisodeListInfo__summary--Jd1WG')
        summary = summary_tag.text.strip() if summary_tag else "줄거리 없음"
        summaries.append(summary)

        # 이미지
        img_tag = soup.select_one('.Poster__image--d9XTI')
        img_url = img_tag['src'] if img_tag else "이미지 없음"
        images.append(img_url)

        # 연령등급
        try:
            age = soup.select_one("em.ContentMetaInfo__info_item--utGrf  span.ContentMetaInfo__dot--uCVnt").next_sibling.strip()
        except:
            age = "연령 정보 없음"
        age_ratings.append(age)

    except Exception as e:
        print(f"오류 발생: {link} → {e}")
        summaries.append("오류 발생")
        images.append("오류 발생")
        age_ratings.append("오류 발생")

driver.quit()

webtoon_df = pd.DataFrame({
    '제목': title_list,
    '작가': author_list,
    '링크': link_list,
    '줄거리': summaries,
    '썸네일': images,
    '연령등급': age_ratings
})

성인 웹툰 감지됨: https://comic.naver.com/webtoon/list?titleId=840184&tab=tue
성인 웹툰 감지됨: https://comic.naver.com/webtoon/list?titleId=836361&tab=tue


In [13]:
webtoon_df.head()

,제목,작가,링크,줄거리,썸네일,연령등급
0,괴력 난신,매드버드 / 김태형 / 한중월야,https://comic.naver.com/webtoon/list?titleId=8...,"살인에서 삶의 즐거움을 느끼는 '겸살귀', 정(正).\n대 연목검장의 셋째공자 목경...",https://image-comic.pstatic.net/webtoon/821597...,15세 이용가
1,김부장,"박만사, 남자의 이야기 / 정종택",https://comic.naver.com/webtoon/list?titleId=7...,“제발 안경 쓴 아저씨는 건들지 말자…”\n오직 자신의 딸 '민지'를 위해 특수요원...,https://image-comic.pstatic.net/webtoon/783053...,15세 이용가
2,우렉 마지노,SIU,https://comic.naver.com/webtoon/list?titleId=8...,<신의 탑> 최초의 외전. 펜타미넘을 쫓아 탑에 들어온 '우렉 마지노' \n그의 선...,https://image-comic.pstatic.net/webtoon/838432...,12세 이용가
3,절대회귀,JP / 박진환 / 장영훈,https://comic.naver.com/webtoon/list?titleId=8...,"""나를 과거로 보내주시오.""\n이 시대에서 그를 죽일 수 없다면, 과거로 돌아가면 ...",https://image-comic.pstatic.net/webtoon/828715...,15세 이용가
4,서울 자가에 대기업 다니는 김 부장 이야기,명랑 / 김병관 / 송희구,https://comic.naver.com/webtoon/list?titleId=8...,"집에서는 과묵하지만 누구보다 아내와 자식을 생각하고, \n회사에서는 책임감 있는 팀...",https://image-comic.pstatic.net/webtoon/819929...,15세 이용가


In [17]:
webtoon_df.to_csv('webtoons_tue.csv', encoding='utf-8-sig', index=False)

In [ ]:
import pymysql

conn = pymysql.connect(
    host='localhost', user='root', password='1234',  db='my_db',
    charset='utf8'
) 

cursor = conn.cursor()

for row in webtoon_df.itertuples(index=False):
    sql = """
    INSERT INTO webtoons (title, author, link, summary, thumbnail, age_rating)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    cursor.execute(sql, (
        row.제목,
        row.작가,
        row.링크,
        row.줄거리,
        row.썸네일,
        row.연령등급
    ))

conn.commit()
cursor.close()
conn.close()